In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt

from mne_bids import BIDSPath

import pandas as pd

plt.rcParams['svg.fonttype'] = 'none'

In [37]:
df = pd.read_csv("../../monkey/stimuli/monkey_util-whisperx_transcript.csv")
len(df)

4794

In [6]:
# get coords
edf_path = BIDSPath(root="../../monkey/derivatives/ecogprep", datatype="ieeg", description="highgamma", extension=".fif")

all_coords = []
for raw_fif in edf_path.match():
    print(raw_fif)
    raw = mne.io.read_raw_fif(raw_fif, verbose=False)
    raw = raw.pick("ecog", exclude="bads")
    ch2loc = {ch['ch_name']: ch['loc'][:3] for ch in raw.info['chs']}
    sub_coords = np.vstack([ch2loc[ch] for ch in raw.info['ch_names']])

    all_coords.append(sub_coords)

../../monkey/derivatives/ecogprep/sub-01/ieeg/sub-01_task-monkey_desc-highgamma_ieeg.fif
../../monkey/derivatives/ecogprep/sub-02/ieeg/sub-02_task-monkey_desc-highgamma_ieeg.fif
../../monkey/derivatives/ecogprep/sub-03/ieeg/sub-03_task-monkey_desc-highgamma_ieeg.fif
../../monkey/derivatives/ecogprep/sub-04/ieeg/sub-04_task-monkey_desc-highgamma_ieeg.fif
../../monkey/derivatives/ecogprep/sub-05/ieeg/sub-05_task-monkey_desc-highgamma_ieeg.fif
../../monkey/derivatives/ecogprep/sub-06/ieeg/sub-06_task-monkey_desc-highgamma_ieeg.fif
../../monkey/derivatives/ecogprep/sub-07/ieeg/sub-07_task-monkey_desc-highgamma_ieeg.fif
../../monkey/derivatives/ecogprep/sub-08/ieeg/sub-08_task-monkey_desc-highgamma_ieeg.fif
../../monkey/derivatives/ecogprep/sub-09/ieeg/sub-09_task-monkey_desc-highgamma_ieeg.fif


In [67]:
channels_tsv = BIDSPath(root="../../monkey", datatype='ieeg', suffix='electrodes', extension='.tsv')
dfs = []
for channel_path in channels_tsv.match():
    df = pd.read_csv(channel_path, sep='\t')
    df['hemi'] = df['x'].apply(lambda x: 'RH' if x > 0 else 'LH')
    df.insert(0, 'subject', channel_path.subject)
    dfs.append(df)
df = pd.concat(dfs)
df

,subject,name,x,y,z,size,group,hemi
0,01,G2,60.000000,-1.333333,-29.333333,NaN,G,RH
1,01,G3,66.000000,-9.333333,-21.333333,NaN,G,RH
2,01,G4,69.000000,-16.000000,-14.000000,NaN,G,RH
3,01,G5,70.666667,-24.666667,-9.000000,NaN,G,RH
4,01,G6,71.000000,-32.000000,-2.000000,NaN,G,RH
...,...,...,...,...,...,...,...,...
183,09,DPMT4,-45.600000,-21.800000,-0.800000,NaN,D,LH
184,09,DPMT5,-50.750000,-22.250000,-1.250000,NaN,D,LH
185,09,DPMT6,-54.000000,-19.500000,-1.500000,NaN,D,LH
186,09,DPMT7,-58.285714,-16.000000,0.285714,NaN,D,LH


In [68]:
sub_hemi = df.groupby(['subject', 'hemi']).name.count()
sub_hemi = pd.pivot(sub_hemi.reset_index(), index='subject', columns='hemi')
sub_hemi

name       
hemi        LH     RH
subject              
01         NaN  103.0
02        95.0    NaN
03       213.0   51.0
04        77.0   78.0
05       159.0    1.0
06       171.0    NaN
07       119.0    NaN
08        75.0    NaN
09       188.0    NaN

In [69]:
sub_elec_group = df.groupby(['subject', 'group']).name.count()
sub_elec_group = pd.pivot(sub_elec_group.reset_index(), index='subject', columns='group')
sub_elec_group

name                   
group       D    EG      G     S
subject                         
01       16.0   NaN   63.0  24.0
02       52.0   NaN    NaN  43.0
03       72.0  64.0   63.0  65.0
04       76.0   NaN    NaN  79.0
05       42.0   NaN  110.0   8.0
06       36.0  64.0   63.0   8.0
07       24.0   NaN   63.0  32.0
08       20.0   NaN   47.0   8.0
09       24.0  64.0   60.0  40.0

In [70]:
pd.concat((sub_hemi, sub_elec_group), axis=1).fillna('n/a')

name                                
            LH     RH     D    EG      G     S
subject                                       
01         n/a  103.0  16.0   n/a   63.0  24.0
02        95.0    n/a  52.0   n/a    n/a  43.0
03       213.0   51.0  72.0  64.0   63.0  65.0
04        77.0   78.0  76.0   n/a    n/a  79.0
05       159.0    1.0  42.0   n/a  110.0   8.0
06       171.0    n/a  36.0  64.0   63.0   8.0
07       119.0    n/a  24.0   n/a   63.0  32.0
08        75.0    n/a  20.0   n/a   47.0   8.0
09       188.0    n/a  24.0  64.0   60.0  40.0